# Daily August 2009 SSH  

In [1]:
  %matplotlib inline

  import matplotlib
  import numpy as np
  import matplotlib.pyplot as plt
  from matplotlib.colors import BoundaryNorm
  from matplotlib.ticker import MaxNLocator
  from netCDF4 import Dataset

  import numpy as np
  import Nio,Ngl
  import os,sys
  import datetime


In [2]:
  path_to_output = "/scratch/rhe/GoM/Data-Assimilation/jan2009c/"
  root = "/scratch/rhe/GoM/Data-Assimilation/jan2009c/"
  latbound_s  = 20 #21
  latbound_n  = 30 #29
  lonbound_w  = -93 #-91
  lonbound_e  = -80 #-82


  instances = 33
  filesi=list(range(1,instances+1,1))
  filestringsi=[str(item).zfill(4) for item in filesi]

  #-- open file
  f = Nio.open_file(path_to_output+"instance_0001/roms_posterior_0001_54939.nc","r")
  gridf = Nio.open_file(path_to_output+"instance_0001/useast_ini_0001.nc","r")
  lats=gridf.variables['lat_rho'][:,0]
  lons=gridf.variables['lon_rho'][0,:]
  wks_type = "png"
  wks = Ngl.open_wks(wks_type,"postage_ssh")
  Ngl.define_colormap(wks,"BlueYellowRed")         # Change color map.

  # Start date and end date
  start_date=datetime.datetime(2009,8,2)
  end_date=datetime.datetime(2009,8,31) # Fig 4.5-9

  #############################################################
  days_since=datetime.datetime(1858,11,17)
  start_date_ordinal=datetime.date.toordinal(start_date)
  end_date_ordinal=datetime.date.toordinal(end_date)
  days_since_ordinal=datetime.date.toordinal(days_since)
    
  start_date_index=start_date_ordinal-days_since_ordinal
  end_date_index=end_date_ordinal-days_since_ordinal
    
  # Optional sanity check
  #start_date_string=datetime.date.fromordinal(days_since_ordinal+start_date_index)
  nfiles=int((end_date_index-start_date_index+1)/2)
  filesi=list(range(start_date_index,end_date_index,2))
  filestringst=[str(item).zfill(4) for item in filesi]
  mres                     = Ngl.Resources()
  mres.mpLimitMode           = "LatLon"   # Specify area of map
  mres.mpMaxLatF             = latbound_n       # to zoom in on.
  mres.mpMinLatF             = latbound_s
  mres.mpMaxLonF             = lonbound_e        # to zoom in on.
  mres.mpMinLonF             = lonbound_w
  mres.sfXArray             = lons         # Area of map to overlay 
  mres.sfYArray             = lats        # contours on.
  mres.mpProjection        = "Mercator"    # choose projection
  mres.tfDoNDCOverlay      = True                  # native grid, no transform
  mres.nglDraw             = False                 # don't draw yet
  mres.nglFrame            = False                 # don't advance frame yet
  mres.pmTickMarkDisplayMode = "Always"            # turn on tickmarks    

  res = Ngl.Resources()
  for t in dir(mres):
   if (t[0:2] == 'cn' or t[0:2] == 'sf' or t[0:3] == 'ngl'):
    setattr(res,t,getattr(mres,t))

  res.mpLimitMode           = "LatLon"   # Specify area of map

  res.sfXArray             = lons         # Area of map to overlay 
  res.sfYArray             = lats         # contours on.
  #plot_base = Ngl.map(wks,mres)
  #res.nglSpreadColorStart  = 99
  #res.nglSpreadColorEnd    = 2
  res.cnFillOn = True
  res.mpLimitMode           = "LatLon"   # Specify area of map
  res.mpMaxLatF             = latbound_n       # to zoom in on.
  res.mpMinLatF             = latbound_s
  res.mpMaxLonF             = lonbound_e        # to zoom in on.
  res.mpMinLonF             = lonbound_w  
  res.cnLevelSelectionMode = "ExplicitLevels" # explicit contour levels
  res.cnLevels             = 0.17             # which level(s) to plot
  res.cnInfoLabelOn        = False            # no info label
  res.cnLineLabelsOn       = False            # no line labels
  res.cnLineThicknessF     = 4                # thickness of contour lines
  res.cnLineColor = -1
  res.pmLabelBarDisplayMode = "Never"
  res.cnLevelSelectionMode = "ManualLevels"  # Select contour levels.
  res.cnMinLevelValF       = -0.4
  res.cnMaxLevelValF       = 0.4
  res.cnLevelSpacingF      =   0.02
  res.mpGridLineColor = -1
  plot = []

  panelres = Ngl.Resources()
  panelres.pmLabelBarOrthogonalPosF = 0.02
  panelres.nglPanelLabelBar                 = True     # Turn on panel labelbar
  panelres.nglPanelLabelBarLabelFontHeightF = 0.015    # Labelbar font height
  panelres.nglPanelLabelBarHeightF          = 0.1350   # Height of labelbar
  panelres.nglPanelLabelBarWidthF           = 0.700    # Width of labelbar
  panelres.lbLabelFont                      = "helvetica-bold" # Labelbar font
  panelres.nglPanelTop                      = 0.925
  panelres.nclPanelBottom                   = 0.07
  
  
  paneli=list(range(1,instances+3,1))
  paneli[33]="pr"
  paneli[34]="po"
  panelstrings=[str(item) for item in paneli]

  panelres.nglPanelFigureStrings            =  panelstrings
  panelres.nglPanelFigureStringsJust        = "BottomRight"  
  panelres.nglPanelFigureStringsFontHeightF = 0.01
  nplots=35

  textres               = Ngl.Resources()

  for t in range(0,nfiles):
    plot = []
    date_obj=datetime.date.fromordinal(int(filestringst[t])+days_since_ordinal)        
    date_str=date_obj.strftime('%b %e, %Y')
    print(date_str)
    for i in range(0,instances):
      #print(root+"instance_"+filestringsi[i]+"/roms_his_"+filestringsi[i]+"_"+filestringst[t]+".nc")
      f = Nio.open_file(root+"instance_"+filestringsi[i]+"/roms_his_"+filestringsi[i]+"_"+filestringst[t]+".nc","r")
      z = f.variables['zeta'][0,:,:]
    
      if i==0:
        zmean = z[:,:]
      else:
        zmean[:,:] = zmean[:,:] + z[:,:]
    
      plot.append(Ngl.contour_map(wks,z,res))
        
        
    zmean = zmean/instances
    plot.append(Ngl.contour_map(wks,zmean,res))
    f = Nio.open_file(root+"output_mean."+filestringst[t]+".nc","r")
    zpost = f.variables['zeta'][0,:,:]
    plot.append(Ngl.contour_map(wks,zpost,res))

    textres.txFontHeightF = 0.025   # Size of title.
    #Ngl.draw_ndc_grid(wks)
    Ngl.text_ndc(wks,date_str,0.5,.83,textres)
    Ngl.panel(wks,plot[0:nplots],[5,7],panelres)    

    #from IPython.display import Image
    #Image("postage_ssh.000001.png ")


Aug  2, 2009
Aug  4, 2009
Aug  6, 2009
Aug  8, 2009
Aug 10, 2009
Aug 12, 2009
Aug 14, 2009
Aug 16, 2009
Aug 18, 2009
Aug 20, 2009
Aug 22, 2009
Aug 24, 2009
Aug 26, 2009
Aug 28, 2009
Aug 30, 2009
